# Прогнозирование временных рядов на карте
## Неделя 7: Оформление проекта

***

### [0] Импорт библиотек и загрузка данных

In [1]:
import pandas as pd
import numpy as np
import folium
import matplotlib.cm as cm
import matplotlib.colors as mpl_colors
from preproc import *

from ipywidgets import widgets
from IPython.display import display
import matplotlib.dates as mdates

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, to_hex
from matplotlib.colors import LogNorm
%matplotlib inline

import geopandas as gpd
from shapely.geometry.polygon import Polygon

Факт + прогноз на Май-Июнь 2016 по 102 регионам:

In [2]:
df_geo = preprocess_data()

In [3]:
C102LOC = (40.7474, -73.8695)    # 90 St-Elmhurst Av Station (~ center of 102 regions)

***
***

### [1] Карты с визуализацией реального и прогнозируемого спроса на такси в выбираемый пользователем момент времени

##### Функция для отображения карт

In [4]:
def choropleth(geo, col):
    
    choropleth = folium.GeoJson(geo, style_function=lambda cell: {
        'fillOpacity': 0.5,
        'weight' : 1,
        'fillColor': clrs.to_hex(cm.viridis_r(int(cell['properties'][col]))),
        }, highlight_function=lambda cell: {
        'weight': 2,
        'fillOpacity': 0.7
    })
    
    popups=[]
    for row in geo.iterrows():
        string = '{:.0f} trips'.format(row[1][col])
        popup = folium.features.RectangleMarker(
            row[1]['geometry'].bounds[::-1], 
            color = 'blue', fill_color = 'white',
            popup = '<div>'+string+'</div>', fill_opacity=0.01
        )
        popups.append(popup)
    
    return choropleth, popups

def plot_choropleth(dind):
    
    dtyme = dind
    
    print('='*26)
    print('|'+'NOW: '+str(dtyme)+'|')
    print('='*26)
    print()
    print('CLICK ON CELLS TO SEE COUNTS:')
    
    geo = df_geo[df_geo.time == dtyme][['region','pred_counts','counts','geometry']]
    geo.crs = {'init' :'epsg:4326'}
    
    fig = folium.Figure(figsize=(16, 7))
    map_pred = folium.Map(width='49%', location=[C102LOC[0], C102LOC[1]], 
                      tiles='CartoDB positron', zoom_start=11)
    map_fact = folium.Map(width='49%', left='50%', position='absolute', location=[C102LOC[0], C102LOC[1]], 
                      tiles='CartoDB positron', zoom_start=11)
    
    map_pred.add_to(fig)
    map_fact.add_to(fig)

    folium.map.Marker([C102LOC[0]+0.11,C102LOC[1]+0.05], icon=folium.DivIcon(
        icon_size=(150,36),icon_anchor=(0,0), html='<div style="font-size: 14pt">PREDICTED TRIPS</div>')).add_to(map_pred)

    folium.map.Marker([C102LOC[0]+0.11,C102LOC[1]+0.05], icon=folium.DivIcon(
        icon_size=(150,36),icon_anchor=(0,0), html='<div style="font-size: 14pt">ACTUAL TRIPS</div>')).add_to(map_fact);
    
    choropleth_pred = choropleth(geo, 'pred_counts')
    choropleth_fact = choropleth(geo, 'counts')
    
    choropleth_pred[0].add_to(map_pred)
    choropleth_fact[0].add_to(map_fact)
    
    for popup in choropleth_pred[1]:
        popup.add_to(map_pred)
    for popup in choropleth_fact[1]:
        popup.add_to(map_fact)
        
    return fig

##### Виджет

In [5]:
dates = pd.date_range(start=df_geo.time.iloc[0],end=df_geo.time.iloc[-1], freq='h')
drange = widgets.interact(plot_choropleth, dind=widgets.SelectionSlider(options=dates,description='DATETIME',
                                                                       continuous_update=False))
drange.widget.children[0].layout.width = '95%'
drange.widget.children[0].description = 'DATETIME'

interactive(children=(SelectionSlider(continuous_update=False, description='DATETIME', options=(Timestamp('2016-05-01 00:00:00', freq='H'), Timestamp('2016-05-01 01:00:00', freq='H'), Timestamp('2016-05-01 02:00:00', freq='H'), Timestamp('2016-05-01 03:00:00', freq='H'), Timestamp('2016-05-01 04:00:00', freq='H'), Timestamp('2016-05-01 05:00:00', freq='H'), Timestamp('2016-05-01 06:00:00', freq='H'), Timestamp('2016-05-01 07:00:00', freq='H'), Timestamp('2016-05-01 08:00:00', freq='H'), Timestamp('2016-05-01 09:00:00', freq='H'), Timestamp('2016-05-01 10:00:00', freq='H'), Timestamp('2016-05-01 11:00:00', freq='H'), Timestamp('2016-05-01 12:00:00', freq='H'), Timestamp('2016-05-01 13:00:00', freq='H'), Timestamp('2016-05-01 14:00:00', freq='H'), Timestamp('2016-05-01 15:00:00', freq='H'), Timestamp('2016-05-01 16:00:00', freq='H'), Timestamp('2016-05-01 17:00:00', freq='H'), Timestamp('2016-05-01 18:00:00', freq='H'), Timestamp('2016-05-01 19:00:00', freq='H'), Timestamp('2016-05-01 20:00:00', freq='H'), Timestamp('2016-05-01 21:00:00', freq='H'), Timestamp('2016-05-01 22:00:00', freq='H'), Timestamp('2016-05-01 23:00:00', freq='H'), Timestamp('2016-05-02 00:00:00', freq='H'), Timestamp('2016-05-02 01:00:00', freq='H'), Timestamp('2016-05-02 02:00:00', freq='H'), Timestamp('2016-05-02 03:00:00', freq='H'), Timestamp('2016-05-02 04:00:00', freq='H'), Timestamp('2016-05-02 05:00:00', freq='H'), Timestamp('2016-05-02 06:00:00', freq='H'), Timestamp('2016-05-02 07:00:00', freq='H'), Timestamp('2016-05-02 08:00:00', freq='H'), Timestamp('2016-05-02 09:00:00', freq='H'), Timestamp('2016-05-02 10:00:00', freq='H'), Timestamp('2016-05-02 11:00:00', freq='H'), Timestamp('2016-05-02 12:00:00', freq='H'), Timestamp('2016-05-02 13:00:00', freq='H'), Timestamp('2016-05-02 14:00:00', freq='H'), Timestamp('2016-05-02 15:00:00', freq='H'), Timestamp('2016-05-02 16:00:00', freq='H'), Timestamp('2016-05-02 17:00:00', freq='H'), Timestamp('2016-05-02 18:00:00', freq='H'), Timestamp('2016-05-02 19:00:00', freq='H'), Timestamp('2016-05-02 20:00:00', freq='H'), Timestamp('2016-05-02 21:00:00', freq='H'), Timestamp('2016-05-02 22:00:00', freq='H'), Timestamp('2016-05-02 23:00:00', freq='H'), Timestamp('2016-05-03 00:00:00', freq='H'), Timestamp('2016-05-03 01:00:00', freq='H'), Timestamp('2016-05-03 02:00:00', freq='H'), Timestamp('2016-05-03 03:00:00', freq='H'), Timestamp('2016-05-03 04:00:00', freq='H'), Timestamp('2016-05-03 05:00:00', freq='H'), Timestamp('2016-05-03 06:00:00', freq='H'), Timestamp('2016-05-03 07:00:00', freq='H'), Timestamp('2016-05-03 08:00:00', freq='H'), Timestamp('2016-05-03 09:00:00', freq='H'), Timestamp('2016-05-03 10:00:00', freq='H'), Timestamp('2016-05-03 11:00:00', freq='H'), Timestamp('2016-05-03 12:00:00', freq='H'), Timestamp('2016-05-03 13:00:00', freq='H'), Timestamp('2016-05-03 14:00:00', freq='H'), Timestamp('2016-05-03 15:00:00', freq='H'), Timestamp('2016-05-03 16:00:00', freq='H'), Timestamp('2016-05-03 17:00:00', freq='H'), Timestamp('2016-05-03 18:00:00', freq='H'), Timestamp('2016-05-03 19:00:00', freq='H'), Timestamp('2016-05-03 20:00:00', freq='H'), Timestamp('2016-05-03 21:00:00', freq='H'), Timestamp('2016-05-03 22:00:00', freq='H'), Timestamp('2016-05-03 23:00:00', freq='H'), Timestamp('2016-05-04 00:00:00', freq='H'), Timestamp('2016-05-04 01:00:00', freq='H'), Timestamp('2016-05-04 02:00:00', freq='H'), Timestamp('2016-05-04 03:00:00', freq='H'), Timestamp('2016-05-04 04:00:00', freq='H'), Timestamp('2016-05-04 05:00:00', freq='H'), Timestamp('2016-05-04 06:00:00', freq='H'), Timestamp('2016-05-04 07:00:00', freq='H'), Timestamp('2016-05-04 08:00:00', freq='H'), Timestamp('2016-05-04 09:00:00', freq='H'), Timestamp('2016-05-04 10:00:00', freq='H'), Timestamp('2016-05-04 11:00:00', freq='H'), Timestamp('2016-05-04 12:00:00', freq='H'), Timestamp('2016-05-04 13:00:00', freq='H'), Timestamp('2016-05-04 14:00:00', freq='H'), Timestamp('2016-05-04 15:00:00', freq='H'), Timestamp('2016-05-04 16:00:00',

***
***

### [2] Временной ряд фактического и прогнозируемого спроса на такси в выбираемой области

##### Функция для построения графика

In [6]:
plt.style.use('bmh')

def plot_region(reg):
    fig, ax = plt.subplots(figsize=(16,6))

    loc = mdates.WeekdayLocator()
    fmt = mdates.DateFormatter('%d-%m-%Y')

    ax.set_title('Trip count in region '+str(reg)+'\n (May 1 - June 30, 2016)')
    ax.set_ylabel('Trip count')
    ax.set_xlabel('Period')
    ax.set_xlim([pd.datetime(2016, 5, 1), pd.datetime(2016, 6, 30)])
    ax.xaxis.set_major_locator(loc)
    ax.xaxis.set_major_formatter(fmt)

    ax.plot_date(x=df_geo[df_geo.region==reg].time,y=df_geo[df_geo.region==reg].pred_counts,
                 ls='solid', lw='1.5', ms=0, label='PREDICTED',  color='r', alpha=1, 
                 solid_capstyle='round', solid_joinstyle = 'round')
    
    ax.plot_date(x=df_geo[df_geo.region==reg].time,y=df_geo[df_geo.region==reg].counts,
                 ls='solid', lw='1.5', ms=0, label='ACTUAL', color='green', alpha=1, 
                 solid_capstyle='round', solid_joinstyle = 'round')

    fig.autofmt_xdate(rotation=45)
    ax.legend(loc='upper right')

##### Виджет

In [7]:
regions = df_geo.region.unique()
rrange = widgets.interact(plot_region, reg=widgets.SelectionSlider(options=regions,description='REGION', 
                                                                   continuous_update=False))
rrange.msg_throttle = 1
rrange.widget.children[0].layout.width = '95%'
rrange.widget.children[0].description = 'REGION'

interactive(children=(SelectionSlider(continuous_update=False, description='REGION', options=(1075, 1076, 1077, 1125, 1126, 1127, 1128, 1129, 1130, 1131, 1132, 1172, 1173, 1174, 1175, 1176, 1177, 1178, 1179, 1180, 1181, 1182, 1183, 1184, 1221, 1222, 1223, 1224, 1225, 1227, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1272, 1273, 1274, 1278, 1279, 1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287, 1326, 1327, 1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1376, 1377, 1378, 1380, 1382, 1383, 1384, 1385, 1386, 1387, 1388, 1389, 1390, 1426, 1431, 1434, 1435, 1436, 1437, 1438, 1439, 1441, 1442, 1480, 1482, 1483, 1530, 1532, 1533, 1580, 1630, 1684, 1733, 1734, 1783, 2068, 2069, 2118, 2119, 2168), value=1075), Output()), _dom_classes=('widget-interact',))